In [ ]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
%load_ext autoreload
%autoreload 2
from IPython import get_ipython; ipython = get_ipython()
import sys, os
from os.path import join as pjoin
import seaborn as sns
p = pjoin( os.path.expandvars('$CODE_MEMORY_ERRORS'), 'state-space-adaptation' )
sys.path.append(p)
sys.path.append(os.path.expandvars('$CODE_MEMORY_ERRORS'))
scripts_dir = pjoin( os.path.expandvars('$CODE_MEMORY_ERRORS'))
script_name = pjoin(scripts_dir,'taumodels/fit_Tan.py')

from config2 import subjects as subjects_NIH
from figure.plots import relplot_multi
from config2 import path_fig
import warnings, gc
from behav_proc import aggRows
from datetime import datetime

import numpy as np
#np.show_config()

from taumodels.models import fitTanModel,fitDiedrischenModel, calcH_min

In [ ]:
from config2 import path_data
import pandas as pd
fnf_NIH = pjoin(path_data,'df_all_multi_tsz_.pkl.zip')
df_all_multi_tsz = pd.read_pickle(fnf_NIH)
dfcc_all_NIH_ = df_all_multi_tsz.query('trial_shift_size == 1 and trial_group_col_calc == "trialwe" '
                           ' and retention_factor_s == "0.924"').copy().sort_values(['subject','trials'])

from datetime import datetime
print('Data mtime',datetime.fromtimestamp(os.stat(fnf_NIH).st_mtime))
dfcc_all_NIH = dfcc_all_NIH_.copy()

assert not dfcc_all_NIH.duplicated(['subject','trial_index']).any()

In [ ]:
dfcc_all_NIH_['error_deg_pscadj'] = dfcc_all_NIH_['error_pscadj'] * 180 / np.pi
coln_error_ = 'error_deg_pscadj'

dfcc_all_NIH_tofit = dfcc_all_NIH_.copy()
bads = dfcc_all_NIH_tofit[coln_error_] > 90
dfcc_all_NIH_tofit.loc[bads,coln_error_] = np.nan

In [ ]:
dfcs_fixhistlen_untrunc = pd.read_pickle(pjoin(path_data,'dfcs_fixhistlen_untrunc.pkl') )
me_pct_excl = pd.read_pickle(pjoin(path_data,'me_pct_excl.pkl') )

In [ ]:
dfcs_fixhistlen_untrunc.columns

In [ ]:
#me_pct_excl['type'] = me_pct_excl['varn'].apply(lambda x: x.split('_')[-1])
me_pct_excl['absed'] = me_pct_excl['varn'].apply(lambda x: x.find('_abs_') >= 0)
me_pct_excl['histlen'] = me_pct_excl['varn'].str.extract(r'(\d+)$', expand=False).astype(int)

In [ ]:
me_pct_excl['statkind']    = me_pct_excl['varn'].str.extract(r'error_pscadj(?<!_abs)_([_a-zA-Z]+)\d+$', expand=False)
#me_pct_excl['type']    = me_pct_excl['varn'].str.extract(r'(?<=_)(.+)', expand=False)
me_pct_excl['statkind'].unique()

In [ ]:
#dfcs1 = dfcs_fixhistlen_untrunc.query('err_sens.abs() <= @ES_thr')  
#truncateDf(dfc1,'err_sens',q=0., infnan_handling = 'discard', cols_uniqify = ['subject'])
import behav_proc
types = ['std','invstd','Tan', 'std_d_mav', 'mav_d_std', 'invTan']
dfcs_fixhistlen, me_pct_excl_ = behav_proc.truncLargeStats(dfcs_fixhistlen_untrunc, me_pct_excl.histlen.unique(), std_mult = 5.,
                                                         suffixes = types, varns0 = ['error_pscadj'])

In [ ]:
display(me_pct_excl)
me_pct_excl['statkind'].unique()

In [ ]:
colns_stat_precomp = me_pct_excl.query('absed == False and type in @types and histlen % 3 == 0').varn.values
colns_stat_precomp

In [ ]:
def d2ps(d):
    s = ''
    for k,v in d.items():
        if isinstance(v,float):
            s += f' {k}={v:.2f},'
        else:
            s += f' {k}={v},'
    return s[:-1]

# Died

In [ ]:
print(coln_error_)

In [ ]:
    dfni = dfcc_all_NIH_[~np.isinf(dfcc_all_NIH_['err_sens'])]
    std_mult = 5
    dfni_d = dfni.groupby(['subject'])['err_sens'].describe().reset_index()
    ES_thr = dfni_d[dfni_d.columns[1:]].mean().to_dict()['std'] * std_mult
    print(f'ES_thr (calc from all) = {ES_thr}')


In [ ]:
bads = dfcc_all_NIH['error_pscadj'] > np.pi / 2
dfcc_all_NIH.loc[bads]

In [ ]:
dfcc_all_NIH['error_pscadj'].describe()

In [ ]:
coln_error_

In [ ]:
coln_pert = 'perturbation_pscadj'
from taumodels.models import interpnan, fitDiedrischenModel

nruns = 8
outd_all = {}
for subj in subjects_NIH:
    outd = {}    
    dfos = dfcc_all_NIH_tofit.query('subject == @subj').copy()        
    perturb = dfos[coln_pert].values
    error = dfos[coln_error_].values
    error = interpnan(error)
        
#     clear_outlier = ES_thr * 5
#     #dfos = dfos.query(qs).copy()
#     inds = dfos.query(
#         f'{coln_error_}.abs() > {clear_outlier}').index
#     dfos.loc[inds,coln_error_] = np.nan    
    
    print('    ', subj, len(dfos))
    outn = 'all_off'; out = fitDiedrischenModel(perturb, error, 
                                     nruns=nruns, online_feedback = False); outd[outn] = out
    outn = 'all_on'; out = fitDiedrischenModel(perturb, error, 
                                     nruns=nruns, online_feedback = True); outd[outn] = out
    
    qsst = 'environment == 0'
    dfos_ = dfos.query(qsst)
    print(len(dfos_))
    perturb = dfos_[coln_pert].values
    error   = dfos_[coln_error_].values
    error = interpnan(error)
    outn = 'stable_off'; out = fitDiedrischenModel(perturb, error, 
                                     nruns=nruns, online_feedback = False); outd[outn] = out
    outn = 'stable_on'; out = fitDiedrischenModel(perturb, error, 
                                     nruns=nruns, online_feedback = True); outd[outn] = out

    qsst = 'environment == 1'
    dfos_ = dfos.query(qsst)
    print(len(dfos_))
    perturb = dfos_[coln_pert].values
    error   = dfos_[coln_error_].values
    error = interpnan(error)
    outn = 'random_off'; out = fitDiedrischenModel(perturb, error, 
                                     nruns=nruns, online_feedback = False); outd[outn] = out
    outn = 'random_on'; out = fitDiedrischenModel(perturb, error, 
                                     nruns=nruns, online_feedback = True); outd[outn] = out
    outd_all[subj] = outd
#output_fboff = out_fboff['output']
#output_fbon  = out_fbon['output']
# -output + perturb

In [ ]:
dfr = []
for subj,outd in outd_all.items():
    for k,out in outd.items():
        d = {}
        A,B,X0 = out['params']
        d['subject'] = subj
        d['outn'] = k        
        
        d['A'] = float(A[0])
        d['B'] = float(B[0])
        d['X0'] = float(X0[0])
        d['online_feedback'] = out['online_feedback']
        d['noises_var'] = out['noises_var']
        d['nruns'] = out['nruns']
        d['asymp'] = out['asymptote']
        d['AIC'] = out['performances'][0]
        d['mse'] = out['performances'][1]
        dfr += [d]
dfr = pd.DataFrame(dfr)

def simRow(row):
    from taumodels.models import interpnan
    from taumodels.state_space_fit_toolbox.state_space_fit import simulate_deterministic_state_space
    subj = row['subject']
    dfos = dfcc_all_NIH_tofit.query('subject == @subj')
    S = np.array([ row['A'],row['B'],row['X0'] ] )
    behav = interpnan(dfos[coln_pert].values) -\
        interpnan(dfos[coln_error_].values)
    #behav = perturb - dofs[]
    r = simulate_deterministic_state_space(S,dfos[coln_pert].values,
                                                  behav,'norm')
    y_pred,state = r
    mse_full = np.mean((y_pred - behav)**2)
    return y_pred,state,mse_full

In [ ]:
def f(row):
    #if 'all' not in row['outn']:
    y_pred,state,mse_full = simRow(row)        
    #else:
    #    mse_full = row['mse']
    return mse_full
        
dfr['mse_full'] = dfr.apply(f,1)
#TODO: test on all, so I have to do like in Herzfeld

In [ ]:
save = 1
if save:
    dfr.to_pickle( pjoin(path_data, 'df_died_fit.pkl') )

In [ ]:
#['outn','mse','A','B']
descr = dfr[['outn','mse_full','mse','A','B']].groupby('outn').describe(percentiles=[])
descr

In [ ]:
descr['A']

In [ ]:
dfr.nlargest(1,['mse_full'])

In [ ]:
d.keys()

In [ ]:
# collect
do_plot = 0
import matplotlib.pyplot as plt
inds_st = np.hstack([np.arange(192),np.arange(192*2,192*3)])
inds_rnd = np.hstack([np.arange(192,192*2),np.arange(192*3,192*4)])

dfcr = []
#for rowi,row in dfr.iloc[0:2].iterrows():
for rowi,row in dfr.iterrows():
    #row = dfr.iloc[3] # 33
    subj = row['subject']
    dfos = dfcc_all_NIH_.query('subject == @subj').copy().sort_values('trial_index')
    dfos_tofit = dfcc_all_NIH_tofit.query('subject == @subj')
    S = np.array([ row['A'],row['B'],row['X0'] ] )
    print(S)

    y_pred,state,mse_full = simRow(row)
    
    behav = interpnan(dfos[coln_pert].values) -\
        interpnan(dfos[coln_error_].values)

    err_pred = dfos[coln_pert].values - y_pred # y_pred - (perturb - err)

    if do_plot:
        plt.figure(figsize=(10,3))
        plt.plot(dfos[coln_error_].values, linewidth=2, label='Observed error') 
        #plt.plot(-output + perturb, linewidth=2, label='Model prediction',alpha=0.8)
        plt.plot(err_pred, linewidth=2, label='Model prediction',alpha=0.8)
        #plt.plot(perturb-error, linewidth=2, label='pert-err') 

        plt.ylabel('Angle (°)')
        plt.xlabel('Trials')
        plt.plot(dfos[coln_pert].values, label='pert', color='pink', ls=':')
        plt.legend() # Call legend function to show legends
        #plt.twinx() # Use twinx function to create a second y-axis
        plt.ylim(-40,60)
        plt.grid(True)

        print('MSE full ',row['mse_full'],mse_full)
        discr = np.abs(err_pred - interpnan(dfos_tofit[coln_error_].values) )**2
        discr_mean = np.mean(discr)
        discr_mean_st = np.mean(discr[inds_st])
        discr_mean_rnd = np.mean(discr[inds_rnd])
        plt.title(f'{subj[:5]} {row["outn"]}  discr={discr_mean:.3f} st={discr_mean_st:.3f} rnd={discr_mean_rnd:.3f}')
        plt.show()

        plt.figure(figsize=(10,2))
        plt.plot(behav,label='behav')
        plt.plot(y_pred,label='state')
        plt.legend()
    
    dfos['state_Died'] = y_pred
    dfos['error_pred_Died'] = err_pred
    #dfos['behav_']
    dfos['mse_full'] = row['mse_full']
    dfos['outn'] = row['outn']
    
#     d = {'state':y_pred, 'behav':behav, 'mse_full':row['mse_full'],
#      'perturbation':dfos['perturbation'].values, 
#      'error':dfos[coln_error_].values,
#     'err_pred':err_pred}
#     d = pd.DataFrame(d)
#     d['subject'] = subj
#     d['trial_index'] = np.arange(len(behav))
#     d['outn'] = row['outn']
    dfcr += [dfos]

dfcr = pd.concat(dfcr).reset_index()

In [ ]:
relplot_multi(data=dfcr,
              ys=[[coln_error_,'error_pred_Died'],[coln_pert,'state_Died']],
             x='trial_index', kind='line');

In [ ]:
d

In [ ]:
relplot_multi?

# Albert

In [ ]:
gc.collect()

In [ ]:
from taumodels.models import fitAlbertModel
from joblib import Parallel, delayed

nruns = 4
dfr = []
fcs = [True,False]

calc_id = 0
args = []
for subj in subjects_NIH:
    dfos = dfcc_all_NIH_tofit.query('subject == @subj')        
    perturb = dfos[coln_pert].values
    error = dfos[coln_error_].values
    error = interpnan(error)
            
    print('   ',subj[:5])
    for fc in fcs:
        for ir in range(nruns):        
            outd = {}                
            
            kw = dict(search_method='norm', force_constraints = fc)
            args += [(calc_id,subj,perturb,error,kw)]
            calc_id += 1

def f(calc_id, subj, perturb, error, kw):
    res,y_pred,state = fitAlbertModel(perturb, error,**kw)
    while not res.success:
        print('Recalc due to non-conv')
        res,y_pred,state = fitAlbertModel(perturb, error,**kw)

    # status, nit, nfev, njev
    d = res.xd.copy()

    print(res)
    d.update({'fun':res.fun, 
              'converged':res.success, 'mess':res.message, 'nit':res.nit,
             'nfev':res.nfev})
    d['subject'] = subj
    d['B_step'] = res.B_step
    d['A_step'] = res.A_step
    d['force_constraints'] = res.force_constraints
    d['calc_id'] = calc_id
    #dfr += [d]
    
    return d,y_pred,state

n_jobs = 20
plr = Parallel(n_jobs=n_jobs, backend='multiprocessing')\
    (delayed(f)(*arg) for arg in args)

dfr = pd.DataFrame( [p[0] for p in plr] )

dfcr = []
for d,y_pred,state in plr:
    subj = d['subject']
    dfos = dfcc_all_NIH_tofit.query('subject == @subj').copy()        
    
    dfos['state1'] = state[:,0]
    dfos['state2'] = state[:,1]
    dfos['state_Albert'] = y_pred
    dfos['error_pred_Albert'] = dfos[coln_pert].values - y_pred
    dfos['B_step'] = res.B_step
    dfos['A_step'] = res.A_step
    dfos['force_constraints'] = res.force_constraints
    dfos['calc_id'] = calc_id # to have corresp between dfr and dfcr later
    dfcr += [dfos]
            
            
#dfr = pd.DataFrame(dfr)
dfcr = pd.concat(dfcr)

In [ ]:
gc.collect()

In [ ]:
dfr

In [ ]:
from behav_proc import aggRows
grp = dfr.groupby(['force_constraints','subject'])
dfr_ = aggRows(dfr,'fun','min',grp)

In [ ]:
dfr.groupby('force_constraints')['fun'].describe()

In [ ]:
dfr_.groupby('force_constraints')['fun'].describe()

In [ ]:
dfr_Albert = dfr_

In [ ]:
dfr_

In [ ]:
dfr_.groupby('force_constraints').describe(percentiles=[])\
    [[('fun','mean'),('A1','mean'),('A2','mean'),('B1','mean'),('B2','mean'),
      ('X01','mean'),('X01','std'),('X02','mean'),('X02','std')]]

In [ ]:
path_data

In [ ]:
dfr_.to_pickle( pjoin(path_data, 'df_albert_fit.pkl') )

In [ ]:
#dfr['fun'].describe()

In [ ]:
relplot_multi(data=dfcr.query('force_constraints == True'),
              ys=[[coln_error_,'error_pred_Albert'],[coln_pert,'state_Albert']],
             x='trial_index', kind='line');

# Tan

In [ ]:
# when calc stat-based ES in the model I could calc using all data (inc outliers)
# or only usining non-outlier data (like I did for stats)

# Read Tan

In [ ]:
del dfr, prl

In [ ]:
gc.collect()
from datetime import datetime
from config2 import path_data, path_fig
import dateutil
#dtstart = dateutil.parser.parse('march 31 2024  18:10')
#dtstart = dateutil.parser.parse('april 2 2024  19:12')

# long list without mser and mses
#dtstart = dateutil.parser.parse('may 12 2024  01:00')

dtstart = dateutil.parser.parse('may 13 2024  09:00')

import glob
prls_fnf = glob.glob(path_data + '/prl_Tan*')

prl = []
for p in prls_fnf:    
    #p=  pjoin(path_data, f'prl_{ind}.npz')
    if not os.path.exists(p):
        continue
    
    dt = datetime.fromtimestamp(os.path.getmtime(p))
    if not dt >= dtstart:
        continue    
    print(p, dt)
    prl_ = np.load(p,allow_pickle=1)['arr_0']
    print(len(prl_))
    
    prl += list(prl_)

print(len(prl))

#print(prl[0].keys())
for i in range(len(prl)):
    prl[i]['prlind'] = i

In [ ]:
parns_calced = ['prlind', 'paramsd','optres', 'mse', 'mser','mses',
                'minfval', 'AIC', 'motor_var'] #'mismatch_var',
parns_arg = ['subject',  'stat_name',  'reg', 'bclip', 
'online_feedback', 'use_true_error', 'use_true_error_stats', 
'stats_powers', 'maxNbTrials', 'minNbTrials', 'maxStartIdx', 
'minStartIdx', 'cmin', 'cmax', 'amin', 'amax',  'rate_is_additive',
            'inds_state_reset','initial_state_bounds','optIC']
parns = parns_calced + parns_arg

parns_unique0 = ['subject',  'stat_name',  'reg', 'bclip', 
'online_feedback', 'use_true_error', 'use_true_error_stats', 
'stats_powers', 'maxNbTrials',  'maxStartIdx', 
 'cmin', 'cmax', 'amin', 'amax', 
'rate_is_additive', 'statkind', 'histlen', 'initial_state_bounds'] 

d0 = prl[0]['paramsd']
opns = list(d0.keys())

In [ ]:
# cols for display
cols = ['subject','mse','mser','mses','A', 'c', 'Bstart', 'X0', 
        'state_reset', 'rate_is_additive','initial_state_bounds',
        'stats_powers',  'use_true_error',
  'rate_is_additive',   
        'StartIdx', 'AIC', 'stat_name',
        'reg', 'motor_var']


In [ ]:
dfr = pd.DataFrame([p for p in prl if p is not None], columns=parns)

In [ ]:
read_saved = 0
if read_saved:
    dfr = pd.read_pickle(pjoin(path_data,'dfr_Tan_calc_res.pkl'))
    dfr['statkind'] = dfr['type']
    dfr = dfr.drop(columns = 'type')

In [ ]:
prl[0].keys()

In [ ]:
#print('optres' in  dfr.columns)
for k in d0.keys():
    dfr[k] = dfr.apply(lambda x: x['paramsd'][k], 1)
dfr['success'] = dfr.apply(lambda x: x['optres'].success, 1)

dfr['absed'] = dfr['stat_name'].apply(lambda x: x.find('_abs_') >= 0 if x is not None else None)
c = ~dfr['stat_name'].isnull()
dfr['histlen'] = None
dfr.loc[c,'histlen'] = dfr.loc[c,'stat_name'].str.extract(r'(\d+)$', expand=False).astype(int)
dfr['statkind'] = None
dfr.loc[c,'statkind']    = dfr.loc[c,'stat_name'].str.extract(r'error_pscadj(?<!_abs)_([_a-zA-Z]+)\d+$', expand=False)
#dfr.loc[c,'statkind']    = dfr.loc[c,'stat_name'].str.extract(r'_([a-zA-Z]+)\d+$', expand=False)
dfr['state_reset'] = dfr['inds_state_reset'].apply(lambda x: len(x))
#me_pct_excl['type']    = me_pct_excl['varn'].str.extract(r'(?<=_)(.+)', expand=False)

print(len(dfr))
dfr_all = dfr
dfr_good = dfr_all.query('success')
print(f'proportion of converged runs {len(dfr_good)/len(dfr)=}')
dfr = dfr_good
from behav_proc import aggRows
#dfr[parns[2:]]

if 'mser' not in prl[0].keys():
    print('no mser')
    #dfr['mser'] = -1
    #dfr['mses'] = -1

parns_unique = parns_unique0 + ['state_reset']

# need to set from None to something otherwise grouping does not work
_inds = dfr.query('use_true_error == 0').index
dfr.loc[_inds,'stat_name'] = dfr.loc[_inds,'stats_powers'].astype('str')
dfr.loc[_inds,'statkind'] = dfr.loc[_inds,'stat_name']
dfr.loc[_inds,'histlen'] = dfr.loc[_inds,'NbPreviousTrials']

In [ ]:
dfr['statkind'].unique()

In [ ]:
assert not dfr.loc[_inds,'NbPreviousTrials'].isna().any()
assert not dfr.loc[_inds,'stats_powers'].isnull().any()
assert not dfr['initial_state_bounds'].isnull().any()

In [ ]:
# add random and stable mse. Pretty long
stable_inds = np.concatenate( [np.arange(192) , np.arange(192*2,192*3) ])
random_inds = np.concatenate( [np.arange(192,192*2) , np.arange(192*3,192*4) ])

if ('mser' not in prl[0].keys()) and ('mser' not in dfr.columns):
    for rowi,row in dfr.iterrows():
    #for rowi,row in dfr.iterrows():
        if rowi % 100 == 0:
            print(rowi, len(dfr))
        subj = row['subject']
        dfos = dfcc_all_NIH_tofit.query('subject == @subj')        
        #perturb = dfos['perturbation_pscadj'].to_numpy()    
        error = dfos[coln_error_].values

        out_cursubj = prl[row['prlind']]
        #adaptationRate = out_cursubj['adaptation_rate']
        output = out_cursubj['error_pred']

        diff = (error - output)**2
        #mse = np.mean(diff)
        mser = np.mean(diff[random_inds])
        mses = np.mean(diff[stable_inds])
        #print(mse,mses,mser, row['mse'])
        dfr.loc[rowi,'mser'] = mser
        dfr.loc[rowi,'mses'] = mses
        #break

In [ ]:
dfr = dfr.drop(columns=['paramsd','optres'])

In [ ]:
save = 0
if save:
    dfr.to_pickle(pjoin(path_data,'dfr_Tan_calc_res2.pkl'))

In [ ]:
#dfr.groupby(parns_unique[:1] + parns_unique[3:]).size()

In [ ]:
dfrb = aggRows(dfr, 'mse', 'min', grp=dfr.groupby(parns_unique))
print(len(dfr),len(dfrb))

In [ ]:
#dfr_ute0[['stat_name','initial_state_bounds','statkind','histlen']]

In [ ]:
# dfr_ute0 = dfr.query('use_true_error == 0')
# for N in range(1,len(parns_unique)):
#     parns_test = parns_unique[:N]
    
# #    parns_test = ['stat_name']
#     print(N,parns_test)
#     dfrb_ute0 = aggRows(dfr_ute0, 'mse', 'min', 
#         grp=dfr_ute0.groupby(parns_test, observed=True))
#     print(len(dfr_ute0),len(dfrb_ute0))

In [ ]:
dfsz = grp.size().to_frame()
dfsz = dfsz.rename(columns={0:'sz'})
dfsz.query('sz == 1')

In [ ]:
dfcc_all_NIH_['perturbation'].abs().max()

In [ ]:
# excl subject
grp = dfrb.groupby(parns_unique[1:]) # except subject
szmn,szmx = grp.size().min(),grp.size().max()
print(szmn,szmx)

grp_ = dfrb.query('use_true_error == 1').groupby(parns_unique[1:]) # except subject
szmn_,szmx_ = grp_.size().min(),grp.size().max()
print(szmn_,szmx_)

assert (20,20) == (szmn,szmx), (szmn,szmx) # both should be 20

In [ ]:
print(parns_unique)

In [ ]:
dfrbme, dfrbstd = grp.mean(numeric_only = True), grp.std(numeric_only = True)
df_ = dfrbme.reset_index().query('histlen > 4')
dfrbmeb = aggRows(df_, 'mse', 'min', 
    grp=df_.groupby(['use_true_error','statkind'])) 

dfrbmeb[['statkind','histlen'] + cols[1:-9]]#['mse']

In [ ]:
df_ = dfrbme.reset_index().query('histlen > 4 and c > 1e-3 and mse <= 150')
dfrbmeb = aggRows(df_, 'mse', 'min', 
    grp=df_.groupby(['use_true_error','statkind','rate_is_additive','state_reset'])) 

dfrbmeb[['statkind','histlen'] + cols[1:-9]]#['mse']

In [ ]:
#ind_to_try = 1627 
ind_to_try = 1067 
row = dfrbmeb.loc[ind_to_try]
print(row)

In [ ]:
dfrb.query('use_true_error == 1').groupby(['stat_name','rate_is_additive']).size().mean()

In [ ]:
# before
print(parns_arg[1:] + opns + parns_calced[2:]  )
#'use_true_error_stats', 'bclip',
dfrb = dfrb.sort_values(parns_arg)
#'cmin', 'cmax', 'amin', 'amax',
dfrb_best = dfrb.nsmallest(5,['mse'])
display(dfrb_best[cols])

dfrb2 = aggRows(dfr, 'mse', 'min', dfr.groupby(['subject']))
dfrb_best2 = dfrb2.nsmallest(8,'mse')
display( dfrb_best2[cols] )

dfrb3 = aggRows(dfr, 'mse', 'min', dfr.query('use_true_error == 1').groupby(['subject']))
dfrb_best3 = dfrb3.nsmallest(5,'mse')
display( dfrb_best3[cols] )


In [ ]:
ind_to_try

In [ ]:
#cols_ = ['use_true_error','histlen','statkind','rate_is_additive','state_reset']
cols_ = parns_unique[1:]
dfrb_ = dfrb.set_index(cols_)
df_ = dfrb_.loc[ tuple(row[cols_].values) ].reset_index()
# df_ = dfrb.query('histlen == @ and statkind == @row["statkind"] '
#                  'and rate_is_additive == @row["rate_is_additive"] '
#                  'and state_reset == @row["state_reset"]')
assert len(df_) == 20, len(df_)
df_['stat_name']

In [ ]:
from matplotlib import pyplot as plt
plot = 0

dfnew = []
for rowi,row in df_.iterrows():
#for rowi,row in dfrb_best3.iterrows():
#for rowi,row in dfr.iterrows():
    subj = row['subject']
    dfos = dfcc_all_NIH_tofit.query('subject == @subj')        
    perturb = dfos['perturbation_pscadj'].to_numpy()    
    error = dfos[coln_error_].values

    out_cursubj = prl[row['prlind']]
    adaptationRate = out_cursubj['adaptation_rate']
    output = out_cursubj['error_pred']
    state = out_cursubj['states']
    
    dfnew += [pd.DataFrame({'error_pred':output, 'error':error, 'perturbation':perturb, 'state':state,
               'trial_index':np.arange(len(error)), 'err_sens':dfos['err_sens'].values, 'err_sens_pred':out_cursubj['adaptation_rate'], 
               'subject':[subj]*len(error)} )]

    if plot:
        plt.figure(figsize=(15,3)) # Use matplotlib.pyplot to plot figures
        ttl = 'Subject {}, optres={}, pows={} onlfb={}, truerr={}, trueerrst={}, rate_add={}, statname={} mse={:.2f}'.format( 
            dfos['subject'].values[0][:5],
            out_cursubj['optres']['success'],out_cursubj['stats_powers'], 
                out_cursubj['online_feedback'],out_cursubj['use_true_error'], 
                out_cursubj['use_true_error_stats'], out_cursubj['rate_is_additive'],
            out_cursubj['stat_name'],
            out_cursubj['mse'] )
        ttl += '\n' + d2ps(out_cursubj['paramsd'])
        plt.title(ttl) # Use string concatenation to make titles
        plt.plot(error, linewidth=2, label='Observed error') # Use label argument to add legends
        plt.plot(output, linewidth=2, label='Model prediction',alpha=0.8)
        plt.plot(state, linewidth=2, label='State', c='purple', alpha=0.6)
        plt.ylabel('Angle (°)')
        plt.xlabel('Trials')
        plt.plot(perturb, label='pert', color='red', ls=':')
        plt.legend() # Call legend function to show legends
        #plt.twinx() # Use twinx function to create a second y-axis
        if adaptationRate is not None:
            Y = 30
            plt.plot(Y + 50*adaptationRate, linewidth=1, color='green')            
            plt.axhline(Y, linewidth=1, label='Y + 50*Adaptation rate', ls=':',color='green')            
            plt.legend() # Call legend function to show legends
        plt.ylim(-40,70)
        #plt.xlim(0,30)
        plt.grid(True)
        plt.show() # Call
dfnew = pd.concat(dfnew)

from figure.plots import relplot_multi
#relplot_multi?

dfnew['errdif'] = dfnew['error'] - dfnew['error_pred']
dfnew.loc[dfnew['trial_index'] <= 3, 'err_sens_pred'] = np.nan

In [ ]:
print(dfnew.columns)

In [ ]:
#perturbation
with warnings.catch_warnings(record=True):
    warnings.filterwarnings("ignore",category=FutureWarning)
    warnings.filterwarnings("ignore",category=UserWarning)        
    fg, df__ = relplot_multi(data=dfnew, kind='line',
    x='trial_index', ys=[['errdif'], ['error','error_pred'],['err_sens_pred']],
             aspect = 2, height=3, ylabel = ['Error discrepancy [deg]','Error [deg]','Simulated error sensitivity'],
              facet_kws = {'sharex':True, 'sharey':False}, ci='sd', ylim=[(-25,25), (-45,45), (0,0.2) ]);
    fg.refline(y=0)    
    fg.refline(x=192)
    fg.refline(x=192*2)
    fg.refline(x=192*3)
    plt.suptitle(df_['stat_name'].iloc[0])
#'Error discrepancy [deg]',

In [ ]:
with warnings.catch_warnings(record=True):
    warnings.filterwarnings("ignore",category=FutureWarning)
    warnings.filterwarnings("ignore",category=UserWarning)        
    relplot_multi(data=dfnew, kind='line',
    x='trial_index', ys=[['perturbation','error','error_pred'],['err_sens_pred'],['err_sens'],['perturbation','state']],
             aspect = 2, height=3);

## debug

In [ ]:
out_cursubj['mse'], out_cursubj['mismatch_var']

In [ ]:
np.mean( (out_cursubj['error_pred'] - error)**2 )

In [ ]:
#fitDiedrischenModel?
from taumodels.state_space_fit_toolbox.state_space_fit import simulate_deterministic_state_space,deterministic_ss_mse,deterministic_ss_mse2
resd = fitDiedrischenModel(perturb, error)
resd['paramsd']

In [ ]:
beh = perturb - error
_arg = resd['params'].flatten(),perturb,beh,'norm'
y_predD,stateD,err_predD = simulate_deterministic_state_space(*_arg)
mse_on_err = np.mean ( (err_predD - error)**2)
mse,mse2 = deterministic_ss_mse(*_arg),deterministic_ss_mse2(*_arg)
mse,mse2,mse_on_err

In [ ]:
# use fitted in Tan but sim using Diedrichsen
opd = out_cursubj['paramsd']
S = np.array( [opd['A'],opd['Bstart'],opd['X0']] )
_arg = S,perturb,beh,'norm'
y_pred,state,err_pred = simulate_deterministic_state_space(*_arg)
mse_on_err = np.mean ( (err_pred - error)**2)
mse,mse2 = deterministic_ss_mse(*_arg),deterministic_ss_mse2(*_arg)
mse,mse2,mse_on_err

In [ ]:
#np.mean( (out_cursubj['error_pred'] - err_pred)**2 )
plt.plot(error)
plt.plot(out_cursubj['error_pred'])
plt.plot(err_pred)
plt.xlim(0,60)


In [ ]:
resH,dH = calcH_min(-perturb, error, None, None, optimize_initial_err_sens='optimize',
    state_retention_bounds=(0,1), weight_update_speed_bounds=(0,0), weight_retention_bounds=(0,0))

In [ ]:
print( dH.keys() )
S = np.array([dH['alpha'], dH['initial_sensitivity'], -dH['initial_state']])
_arg = S,perturb,beh,'norm'
y_predH,state,err_predH = simulate_deterministic_state_space(*_arg)
mse_on_err = np.mean ( (err_predH - error)**2)
mse,mse2 = deterministic_ss_mse(*_arg),deterministic_ss_mse2(*_arg)
mse,mse2,mse_on_err

In [ ]:
plt.plot(y_predD,label='stateD')
plt.plot(y_pred,label='stateT')
plt.plot(resd['states'],label='state_T_T', ls=':',c='r')
plt.legend()
plt.show()

plt.plot(y_predH,label='stateH')
plt.plot(-dH['motor_output'],label='state_H_H')
#plt.plot(y_pred,label='stateT')
#plt.plot(resd['states'],label='state_T_T', ls=':',c='r')
plt.xlim(0,60)
plt.legend()

In [ ]:
#18*400

In [ ]:
#args_all = args_all[18 * N:]

In [ ]:
print('len args = ',len(args_all))
import gc;gc.collect()

## old ver

In [ ]:
startidx, histlen

In [ ]:
import time
from taumodels.models import fitDiedrischenModel,fitTanModel
dfnamesubj2res_Tan = {}
nruns = 20
n_jobs = 32
#reg = 1e-3
#reg = 10
#reg = 5
#reg = 1
#reg = 3
nruns = 5
#nsubj = 20
minStartIdx =3; maxStartIdx=7; maxNbTrials = 11; minNbTrials = 2
#stats_powers = (2,-2)
#stats_powers = (0,-1)
#fit_trials = 'stable' # the problem is how to reset state between to blocks
#use_true_error=0; use_true_error_stats=1
#nsubj=1; nruns=3; reg=3; fit_trials='stable'; minStartIdx =6; maxStartIdx=7; maxNbTrials = 7;minNbTrials = 2
nsubj=3; nruns=4; minStartIdx =6; maxStartIdx=7; maxNbTrials = 6;minNbTrials = 3
#for dfname,df in zip(['NIH', 'cc'], [dfcc_all_NIH,dfcc_all]):
for dfname,df in zip(['NIH'], [dfcc_all_NIH]):
    for subj in df.subject.unique()[:nsubj]:
        print(dfname, '   subj == ',subj)
        qs0 = 'subject == @subj'
        
        motor_var = df.query(qs0 + ' and trial_index >= 10 and trial_index <= 24')['error'].abs().std()
        t0 = time.time()
        inds_state_reset = []  
        
        dfnamesubj2res_Tan[(dfname,subj)]= []
        for (stats_powers,use_true_error,use_true_error_stats,reg,fit_trials) in prod:
            if dfname == 'NIH':
                coln_error_ = 'error_deg'                       
                if fit_trials == 'stable':
                    qs = qs0 + ' and environment == 0'
                    #inds_state_reset = np.arange(192,700, 192)
                    inds_state_reset = np.array([192])
                elif fit_trials == 'random':
                    qs = qs0 + ' and environment == 1'
                    #inds_state_reset = np.arange(192,700, 192)
                    inds_state_reset = np.array([192])
                else:
                    qs = qs0
            else:         
                qs = qs0 + ' and trial_index >= @numtrain'            
                coln_error_ = coln_error
            dfos = df.query(qs)    
            
            
            motor_var_ = motor_var
            if use_true_error:
                motor_var_ = 0. # would just confuse model additionally
            
            out_cursubj = None
            import warnings
            with warnings.catch_warnings(record=True) as w:
                warnings.filterwarnings('ignore',category=RuntimeWarning)

                out_cursubj = fitTanModel(dfos, coln_error_, n_jobs, 
                            bclip=(-1,1), nruns=nruns,
                         reg = reg, neg_error=1,
                        maxNbTrials = maxNbTrials,
                        minNbTrials = minNbTrials,
                        minStartIdx = minStartIdx,
                        maxStartIdx=maxStartIdx, cmax = 0.007, online_feedback=False, 
                        inds_state_reset = inds_state_reset, stats_powers=stats_powers,
                        motor_var = motor_var_,
                        use_true_error=use_true_error, use_true_error_stats=use_true_error_stats)
        
            out_cursubj['fit_trials'] = fit_trials
            dfnamesubj2res_Tan[(dfname,subj)].append( out_cursubj )
            #for subj in subj2out_Tan: 
            #    print('     ',subj)
            #    for k,v in subj2out_Tan[subj ].items():
            print(out_cursubj['params'],stats_powers,use_true_error,use_true_error_stats )

            perturb = dfos['perturbation'].to_numpy()    
            adaptationRate = out_cursubj['adaptation_rate']
            output = out_cursubj['y_pred']
            error = dfos['error_deg'].values

            plot = 0
            if plot:
                plt.figure(figsize=(15,3)) # Use matplotlib.pyplot to plot figures
                ttl = 'Subject {}, optres={}, pows={} onlfb={}, truerr={}, trueerrst={}'.format( 
                    dfos['subject'].values[0][:5],
                    out_cursubj['optres']['success'],out_cursubj['stats_powers'], 
                        out_cursubj['online_feedback'],out_cursubj['use_true_error'], 
                        out_cursubj['use_true_error_stats'] )            
                plt.title(ttl) # Use string concatenation to make titles
                plt.plot(error, linewidth=2, label='Observed error') # Use label argument to add legends
                plt.plot(output, linewidth=2, label='Model prediction',alpha=0.8)
                plt.ylabel('Angle (°)')
                plt.xlabel('Trials')
                plt.plot(perturb, label='pert', color='red', ls=':')
                plt.legend() # Call legend function to show legends
                #plt.twinx() # Use twinx function to create a second y-axis
                if adaptationRate is not None:
                    plt.plot(30 + 50*adaptationRate, linewidth=1, label='Adaptation rate', color='green')            
                    plt.legend() # Call legend function to show legends
                plt.ylim(-40,60)
                plt.grid(True)
                plt.show() # Call

## try one 

In [ ]:
arg0 = args_all[10].copy()
for k,v in arg0.items():
    if not isinstance(v,np.ndarray):
        print(k,v)
#print(arg0)
arg0['use_true_errors']

In [ ]:
np.sum( np.isnan(arg['errors']) )

In [ ]:
#fitmask_vals
#s = int(np.ceil(time.time() * 1e4) ) % int( 1e5 )   
#arg0['seed'] = s
#arg0['fitmask'] = fitmask_vals
r = _opt(**arg0)

In [ ]:
r[1]['fun_full']

In [ ]:
_opt?

In [ ]:
arg0.keys()

In [ ]:
arg0 = args_all[0].copy()
from taumodels.models import interpnan
verbose = 0
arg0sim = arg0.copy()

print('Num nans in errors= ', np.sum(np.isnan(arg0['errors'])))
arg0sim['true_errors'] = interpnan(arg0['errors'])
#arg0sim['true_err'] = arg0['errors']
todel = ['optimize_initial_err_sens','err_sens','errors','fit_to',
        'reg','timeout','fitmask_vals','fitmask',
        'err_handling','subject','rnsuff']
for td in todel:
    del arg0sim[td]
#subj = arg0['subject']
# df0 = dfcc_all_NIH_.query('trial_shift_size == 1 and '
#     'trial_group_col_calc == "trials" and '
#     'retention_factor_s == "0.924" and '
#     'subject == @subj').sort_values(['trials'])
#pre_break_duration = df0['pre_break_duration'].values
#error   = df0['error_pscadj'].values * 180 / np.pi
#perturb = -df0['perturbation_pscadj'].values
#perturb[perturb > 180] = perturb[perturb > 180] - 360 

#errors_ =  interpnan(error.copy())  
#err_sens = interpnan(df0['err_sens'].values.copy())  

#use_true_errors = row['use_true_errors']
#cap_err_sens = row['cap_err_sens']    
r = simMoE(**arg0sim, verbose=verbose)
#-perturb,EC_mask=EC_mask,true_errors=errors_,
#            pre_break_duration=pre_break_duration,
#            **pard, cap_err_sens=cap_err_sens,
#           small_err_thr = row['small_err_thr'],
#           num_bases = row['num_bases'],
#    use_true_errors=use_true_errors

(motor_output,error_pred2,ws,err_sens2, 
    gaussian_centers,gaussian_variances) = r 

from taumodels.models import calcLoss_H
fun = calcLoss_H(arg0sim['true_errors'], error_pred2, arg0['err_sens'], 
                 err_sens2,  None, arg0['fit_to'], arg0['reg'])

err_sens2_ = err_sens2#[1:]
print(err_sens2_.max(), err_sens2_.mean(), err_sens2_.std(),fun)


## run all

In [ ]:
prl = []

In [ ]:
# run
N = 400
ind = 0
i = 0

In [ ]:
n_jobs_for_all = 52
#backend = 'multiprocessing' # to see output realtime
backend = 'loky'

#from IPython.core.output import capture_output
# Capture the output of your code block
#%%capture captured
if n_jobs_for_all == 1:
    for arg in args_all:
        prl += [_opt(**arg)]
else:    
    #plr = []    
    while ind < len(args_all):      
        gc.collect()
        subargs = args_all[ind:ind+N]
        prl_ = Parallel(n_jobs=n_jobs_for_all, backend=backend,
                    )(delayed(_opt)(**arg) for arg in subargs) 
        #plr += [plr_]
        s_ = str(datetime.now())[:-7].replace(' ','_')
        np.savez( pjoin(path_data, f'prl_Herz_{i}__{s_}'), prl_ )

        del prl_
        ind += N
        i += 1

#Nout = 40        
# Access and print the last N lines (replace N with your desired number)
#last_lines = captured.stdout.splitlines()[-Nout:]
#print(*last_lines, sep='\n')            

In [ ]:
print(ind, ind/N)

# read

In [ ]:
from datetime import datetime
from config2 import path_data, path_fig
import dateutil
#dtstart = dateutil.parser.parse('march 31 2024  18:10')
dtstart = dateutil.parser.parse('april 2 2024  19:12')

import glob
prls_fnf = glob.glob(path_data + '/prl_Herz*')

prl = []
for p in prls_fnf:    
    #p=  pjoin(path_data, f'prl_{ind}.npz')
    if not os.path.exists(p):
        continue
    
    dt = datetime.fromtimestamp(os.path.getmtime(p))
    if not dt >= dtstart:
        continue    
    print(p, dt)
    prl_ = np.load(p,allow_pickle=1)['arr_0']
    print(len(prl_))
    
    prl += list(prl_)

print(len(prl))

In [ ]:
from behav_proc import aggRows
import pandas as pd
prl = pd.DataFrame(prl, columns=['minres','pard','arg','subject']) 
for k in prl.arg.values[0]:
    prl[k] = prl.arg.apply(lambda x: x.get(k,None))

def f(x):
    thr = x["small_err_thr"]
    
    r = (f'useTrueErr={x["use_true_errors"]}; optimES0={x["optimize_initial_err_sens"]};'
    f' capES={x["cap_err_sens"]}; '
    f' fit_to={x["fit_to"]}; {x["rnsuff"]}'     )
    return r
#sethr={thr > 1e-6}; 
prl['runname'] = prl.apply(f,1)

###################################################
prlg = prl[~prl.pard.isnull()].copy()

pard0 = prlg.pard.values[0]
for k in pard0.keys():
    if isinstance(pard0[k], np.ndarray):
        continue
    prlg[k] = prlg.pard.apply(lambda x: x.get(k,None))


prlg['fun'] = prlg.apply(lambda x: x['minres'].fun, 1)                   

In [ ]:
#prlg.iloc[0][prlg.columns[3:]]

In [ ]:
prlg.use_true_errors.any()

In [ ]:
#prlg[prlg.columns[3:]]

# aug

In [ ]:
ipython.run_line_magic('run', f'-i {script_name} '
                    f'--subject {subjects_NIH[0]} '                    
                    f'--exit_after init')

In [ ]:
def recalcRow(row, pard_upd = None, verbose=0):
    if row['fitmask'] == 'no':
        prlg.loc[rowi,'fun_full'] = row['fun']
        return np.nan
    runname = row['runname']
    pard_ = row['pard']
    if pard_ is None:
        print('empty',runname)
        return np.nan
    pard = pard_.copy()
    if 'fun' in pard:
        del pard['fun']
        del pard['nit']
        del pard['nfev']        
        
    s = int(np.ceil(time.time() * 1e4) ) % int( 1e5  )
    
    if pard_upd is not None:
        for k in pard_upd.keys():
            if pard[k] != pard_upd[k]:
                print(f'    {k}: {pard[k]} -> {pard_upd[k]}')
        pard.update(pard_upd)
    
    subj = row["subject"]
    print('recalcRow: ',ind, len(prlg), subj, runname)
    df0 = dfcc_all_NIH_.query('trial_shift_size == 1 and '
        'trial_group_col_calc == "trials" and '
        'retention_factor_s == "0.924" and '
        'subject == @subj').sort_values(['trials'])
    pre_break_duration = df0['pre_break_duration'].values
    error   = df0['error_pscadj'].values * 180 / np.pi
    perturb = df0['perturbation_pscadj'].values
    
    print('Num nans in errors= ', np.sum(np.isnan(error)))
    errors_ =  interpnan(error.copy())  
    err_sens = interpnan(df0['err_sens'].values.copy())  

    use_true_errors = row['use_true_errors']
    cap_err_sens = row['cap_err_sens']    
    r = simMoE(-perturb,EC_mask=EC_mask,true_errors=errors_,
                pre_break_duration=pre_break_duration,
                **pard, cap_err_sens=cap_err_sens,
               small_err_thr = row['small_err_thr'],
               num_bases = row['num_bases'],
        use_true_errors=use_true_errors, verbose=verbose)
    
    (motor_output,error_pred2,ws,err_sens2, 
        gaussian_centers,gaussian_variances) = r 
    
    err_sens2g = err_sens2[~np.isnan(err_sens2)]
    print('err_sens2 len,mean,std = ', len(err_sens2g), err_sens2g.mean(), err_sens2g.std())
    # fitmask should be 'no' here
    fun = calcLoss_H(errors_, error_pred2, err_sens, err_sens2,  None, row['fit_to'], row['reg'])
    return fun

In [ ]:
#row['small_err_thr']

In [ ]:
# effecot of weight_forgetting_exp_pause is really small because ES in general changes little

In [ ]:
verbose = 0
row = best_par_rows.iloc[-5]
for i in range(1):    
    v = np.random.uniform(0,2)    
    print(v, row['fun_full'], row['initial_sensitivity'])
    display(row['pard'])
    fun = recalcRow(row, {'weight_forgetting_exp_pause':v}, verbose=verbose)
    print('  fun_full = ',fun, '\n\n')

In [ ]:
best_par_rows[['fun_full','weight_forgetting_exp_pause']]

In [ ]:
# add values for those with fitmask not full
from taumodels.models import simMoE, calcLoss_H, interpnan
prlg['fun_full'] = np.nan
ind = 0 
for rowi,row in prlg.iterrows():
    fun = recalcRow(row)
    prlg.loc[rowi,'fun_full'] = fun
    print('    fun=',fun)
    ind += 1

# best

In [ ]:
# best within runname (across runs)
bestCalcs = aggRows(prlg, 'fun_full', 'min',
     grp = prlg.groupby(['subject','runname'] ) ).reset_index()

bestCalcs['fitmask'] = bestCalcs['runname'].str.extract(r'_mask=([^_]*)')

bestCalcs.to_pickle(pjoin(path_data,'herz_param_calced6.pkl.zip')
    ,compression='zip')

bestCalcs[['fitmask','fun','fun_full']]

In [ ]:
prl.use_true_errors.unique(), prlg.use_true_errors.unique(), bestCalcs.use_true_errors.unique()

In [ ]:
bcs = bestCalcs
ss = sum( [list(b.keys()) for b in bcs['pard'].values], [] )
ss = set(ss ) - set(['error_pred', 'err_sens'])
pardkeys = list(sorted(ss))

def f(row):
    pard = row.get('pard',None)
    if pard is None:
        tpl = tuple( [np.nan] * len(pardkeys))
    else:
        tpl = tuple( [pard[pk] for pk in pardkeys] )
    return tpl
bcs[pardkeys]  = bcs.apply(f, 1 ,result_type='expand')
prlg[pardkeys] = prlg.apply(f, 1 ,result_type='expand')

def f(row):
    pard = row['pard']
    ES = pard['err_sens']
    return np.mean(ES), np.std(ES), np.max(ES)
cols_ESsimstat = ['err_sens_mean','err_sens_std','err_sens_max']
bcs[cols_ESsimstat] = bcs.apply(f,1, result_type='expand')

# param explanation: alpha retetion, alpha w retention of weights, eta weight update speed
grp = bcs.groupby('runname')[pardkeys + cols_ESsimstat] 
me = grp.mean()#.mean(numeric_only = True)
desc = grp.describe()#[('alpha')]

#desc_allsubj = desc[desc[('fun_full','count')] == 20]
desc_allsubj = desc[desc[('fun_full','count')] >= 10]

desc_allsubj = desc_allsubj.sort_values([('fun_full','mean')])

rn_best = desc_allsubj.iloc[0].name
print(rn_best)

In [ ]:
# prep ols
cols = []
for k in pardkeys:
    if k in ['fun','fun_full']:
        continue
    cols += [(k,'mean'),(k,'std') ]

cols_ESsimstat_ext = []
for k in cols_ESsimstat:    
    cols += [(k,'mean'),(k,'std') ]

cols = [('eta','count'), ('fun_full','mean'), ('fun_full','std')] + cols
print(cols)

In [ ]:
desc_allsubj[cols]

In [ ]:
# variation of weight forgetting within runname
best_par_rows = bcs.query('runname == @rn_best')
best_par_rows[['subject','weight_forgetting_exp_pause']]

In [ ]:
#%run

In [ ]:
rn_new = desc_allsubj.reset_index().runname.str.replace('capES=False; ','').\
    str.replace(' optimES0=optimize; ','').str.replace('nb=-1','')
rn_new = rn_new.str.replace('useTrueErr','trueErr').str.replace('__sethr=','; ')
rn_new.values

In [ ]:
desc_allsubj_.columns

In [ ]:
# what is the best runname withing subset?

# alpha ~ 0.95. alpha_w ~ 0.36. eta~0.02, eta=0.05, 
# init ES ~ 0.0064 (std = 0.08), pause forget ~ 0.8 (std=0.87).
grp = bcs.groupby(['runname','use_true_errors','fit_to'])[pardkeys + cols_ESsimstat] 
me = grp.mean()#.mean(numeric_only = True)
grp2 = me.groupby(['use_true_errors','fit_to'])
mebest = aggRows(me, 'fun_full', 'min', grp=grp2).reset_index().sort_values('fun_full')

#desc_allsubj_ = desc_allsubj.copy().reset_index()
rn_new = desc_allsubj.index.str.replace('capES=False; ','').\
    str.replace(' optimES0=optimize; ','').str.replace('nb=-1','')
rn_new = rn_new.str.replace('useTrueErr','trueErr').str.replace('__sethr=','; ')
desc_allsubj['rnshort'] = rn_new

rns_best = list(mebest.runname) 
rns = rns_best

runnames_sorted_funfull = mebest['runname'].values

display(rns_best)
desc_allsubj_ = desc_allsubj.reset_index()
desc_allsubj_ = desc_allsubj_[desc_allsubj_['runname'].isin(rns_best)]
display( desc_allsubj_[[('rnshort','')] + cols[:11]] )
#display( desc_allsubj_[cols[6:11]] )
display( desc_allsubj_[[('rnshort','')] +cols[11:]] )
display( desc_allsubj_[[('rnshort','')] +cols[15:]] )

## old

In [ ]:
# get the very best
# mean across subjects
bc = bestCalcs.groupby(['runname'])['fun_full'].mean().to_frame()
bcstd = bestCalcs.groupby(['runname'])['fun_full'].std().to_frame()
i = bc['fun_full'].idxmin()
print( bc.loc[i].name)
print('mean = ', bc.loc[i,'fun_full'], ', std = ',bcstd.loc[i,'fun_full'] )
rn = bc.loc[i].name

In [ ]:
cols = ['eta','alpha','alpha_w','weight_forgetting_exp_pause','initial_sensitivity','fun']
bc00 = bestCalcs.query('use_true_errors == True and fitmask != "random" and reg > 0')
bc0 = bc00.groupby(['runname'])
bc = bc0['fun'].mean().to_frame()
print(bc)
if len(bc):
    i = bc['fun'].idxmin()
    print( bc.loc[i] )
    rn = bc.loc[i].name

In [ ]:
bad_subjects = set(subjects_NIH) - set( bc00.query('runname == @rn')['subject'].values )
bad_subjects = list(bad_subjects)
print(len(bad_subjects),bad_subjects)

In [ ]:
#rn = 'useTrueErr=True; optimES0=optimize; capES=False; fit_to=errors; _reg=0.25_mask=stable_nb=-1_sethr=mestd*1.0'

In [ ]:
prl.query('subject.isin(@bad_subjects) and runname == @rn').iloc[0].minres

In [ ]:
bc0[cols].mean().nsmallest(10,columns=['fun'])[cols]

In [ ]:
dfd = bestCalcs.query('use_true_errors == True and runname == @rn').\
    groupby(['runname'])[cols].describe(percentiles=[])

In [ ]:
# df_ = dfcc_all_NIH_.query('trial_shift_size == 1 and '
#     'trial_group_col_calc == "trials" and retention_factor_s == "0.924"'
#                          )
# df_.loc[df_['pre_break_duration'] > 0,
#         ['subject','trials','pre_break_duration']]

# plot prep

In [ ]:
assert not bestCalcs.duplicated(['subject','runname']).any()
from taumodels.models import calcLoss_H,interpnan

### old

In [ ]:
# resim
dfcc_all_NIH_['perturbation_pscadj'] = dfcc_all_NIH_['perturbation']
dfcc_all_NIH_.loc[dfcc_all_NIH_['pert_seq_code'] == 1, 
    'perturbation_pscadj'] = -dfcc_all_NIH_['perturbation']

#df0['perturbation'].describe()

dfr =[]
for rowi,row in bestCalcs.iterrows():
    runname = row['runname']
    pard_ = row['pard']
    if pard_ is None:
        print('empty',runname)
        continue
    pard = pard_.copy()
    if 'fun' in pard:
        del pard['fun']
        del pard['nit']
        del pard['nfev']        
    
    subj = row["subject"]
    print(rowi,len(bestCalcs), subj, runname)
    df0 = dfcc_all_NIH_.query('trial_shift_size == 1 and '
        'trial_group_col_calc == "trials" and '
        'retention_factor_s == "0.924" and '
        'subject == @subj').sort_values(['trials'])
    pre_break_duration = df0['pre_break_duration'].values
    error = df0['error_pscadj'].values * 180 / np.pi
    perturb = df0['perturbation_pscadj'].values
    
    errors_ = interpnan(error.copy())  
    #err_sens = np.interpnan(df0['err_sens'])  

    use_true_errors = row['use_true_errors']
    cap_err_sens = row['cap_err_sens']    
    r = simMoE(-perturb,EC_mask=EC_mask,true_errors=errors_,
                pre_break_duration=pre_break_duration,
                **pard, cap_err_sens=cap_err_sens,
               small_err_thr = row['small_err_thr'],
               num_bases = row['num_bases'],
        use_true_errors=use_true_errors)
    
    (motor_output,error_pred2,ws,err_sens2, 
        gaussian_centers,gaussian_variances) = r 
    
    # fitmask should be 'no' here
    #fun = calcLoss_H(errors_, error_pred2, err_sens, err_sens2, 'no', row['fit_to'])
    #bestCalcs.loc[row,'fun_full'] = fun
    
    #if prep_plot:
    _dat = np.array((motor_output,error_pred2,
        err_sens2, np.arange(len(motor_output), dtype=int) ) )
    df_ = pd.DataFrame(data=_dat.T, 
        columns = ['motor_output','error_pred2',
                'err_sens2','trial_index'])
            
    df_['err_sens'] = df0['err_sens'].values
    df_['error_pscadj_deg'] = df0['error_pscadj'].values * 180 / np.pi

    for kn,kv in row.items():
        df_[kn] = kv
    dfr += [df_ ]
dfr = pd.concat(dfr,ignore_index=1)

dfr = dfr.sort_values(['runname'], ascending=False)

runnames = dfr['runname'].unique()
print(runnames)

gc.collect()

In [ ]:
pard0['err_sens']

In [ ]:
bestCalcs.columns

### new

In [ ]:
assert not bestCalcs.duplicated(['subject','runname']).any()
from taumodels.models import calcLoss_H,interpnan

ipython.run_line_magic('run', f'-i {script_name} '
                    f'--subject {subjects_NIH[0]} '                    
                    f'--exit_after init')

In [ ]:
dfr =[]
for rowi,row in bestCalcs.query('runname.isin(@rns_best)').iterrows():
#for rowi,row in bestCalcs.iterrows():    
    pard = row['pard']
    subj = row["subject"]
    
    err_sens_pred   = pard['err_sens']
    error_pred = pard['error_pred']    
    
    print(rowi,len(bestCalcs), subj, row['runname'])
    df0 = dfcc_all_NIH_.query('trial_shift_size == 1 and '
        'trial_group_col_calc == "trials" and '
        'retention_factor_s == "0.924" and '
        'subject == @subj').sort_values(['trials'])    
    
    _dat = np.array((error_pred,
        err_sens_pred, np.arange(len(err_sens_pred), dtype=int) ) )
    df_ = pd.DataFrame(data=_dat.T, 
        columns = ['error_pred2',
                'err_sens2','trial_index'])
            
    df_['err_sens'] = df0['err_sens'].values
    df_['error_pscadj_deg'] = df0['error_pscadj'].values * 180 / np.pi
    
    df_['errdif'] = (df_['error_pred2'] - df_['error_pscadj_deg']).abs()
    
    for kn,kv in row.items():
        if kn in ['pard','arg','minres','err_sens','error_pred']:
            continue
        df_[kn] = kv
    dfr += [df_ ]
dfr = pd.concat(dfr,ignore_index=1)

dfr = dfr.sort_values(['runname'], ascending=False)

runnames = dfr['runname'].unique()
print(runnames)

gc.collect()

In [ ]:
print( prl.subject.nunique(), dfr.subject.nunique(), len(runnames) )

In [ ]:
# # alpha retetion, alpha w retention of weights, eta weight update speed
# grp = bcs.groupby('runname')[pardkeys + ['fun_full','fun']] 
# me = grp.mean()#.mean(numeric_only = True)
# me = me.sort_values(['fun_full'])
# me['fun_full_diff'] = me['fun_full'].diff()
# display(me.iloc[:20])

# Plot 

In [ ]:
gc.collect()

In [ ]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
def row2s(row):
    sres =''
    sres_l = []
    for i,((a1,b1),(a2,b2)) in enumerate( zip(cols[1::2],cols[2::2]) ):
        #print(i, a1,b1, a2,b2)    
        c1,c2 = row[(a1,'mean')],row[(a1,'std')]
        bb = c2/c1 * 100
        a1 = a1.replace('forgetting_exp_pause','forget')
        a1 = a1.replace('initial_sensitivity','ES0')
        s = f'{a1}={c1:.3f} (std={bb:.2f}%)'
        sres += s + '; '
        if len(sres) > 60:
            sres_l += [sres]
            sres = ''
        #print('      ' ,s)
    sres_final = '\n'.join(sres_l) 
    return sres_final
    #print('\n\n')
    #print('\n'.join(sres_l) )

In [ ]:
# plot
#%debug
import warnings
trs = dfos.query('perturbation.diff().abs() > 0 and env == "stable"')['trials'].values

ys = [['errdif'], ['error_pred2','error_pscadj_deg'], ['err_sens2'], ['err_sens']]

for rn in rns_best:
#for rn in list(runnames):#[2:]:
    gc.collect()
    df_ = dfr.query('runname == @rn').copy()# and fit_to == "errors"')
    
    row = desc_allsubj.loc[rn,cols]
    ss = '\n' + row2s(row)
    print(rn,ss)
    
    if len(df_) == 0:
        print('Emtpy for ',rn)
        continue
    with warnings.catch_warnings(record=True):
        warnings.filterwarnings("ignore",category=FutureWarning)
        warnings.filterwarnings("ignore",category=UserWarning)        
        fg,df__ = relplot_multi(kind='line',data=df_,
                x='trial_index',
            ys=ys, 
            ylabel = ['errdiff','error','ES_opt','ES_true'],
            ylim=[(0,40), (-50,50), (-0.01,0.15), (-2,2)], aspect = 2, 
                                height = 3,
            facet_kws = {'sharex':True, 'sharey':False})

        fg.refline(y=0)    
        fg.refline(x=192)
        fg.refline(x=192*2)
        fg.refline(x=192*3)
        fg.axes[2,0].axhline(y=row[('initial_sensitivity')]['mean'],
                            color='grey',ls='--')
        plt.suptitle(rn + '\n' + ss, fontsize=10)    

        for tr in trs:
            fg.refline(x=tr) 
                
                
        plt.savefig(pjoin(path_fig, '_Herz_fit_to_ES', rn + '.pdf'))
        print('   saved')
        plt.close()
        
    del df__,df_
    
    print(rn)

In [ ]:
row

In [ ]:
print(bestCalcs.query('runname == @rn').columns)

In [ ]:
rns_best

In [ ]:
# plot nice
#%debug
import warnings
trs = dfos.query('perturbation.diff().abs() > 0 and env == "stable"')['trials'].values

#ys = [['errdif'], ['error_pred2','error_pscadj_deg'], ['err_sens2'], ['err_sens']]
ys = [['errdif'], ['error_pred2','error_pscadj_deg'], ['err_sens2']]

for rn in np.array(rns_best)[[0,2]]:
#for rn in rns_best[:2]:
#for rn in list(runnames):#[2:]:
    gc.collect()
    df_ = dfr.query('runname == @rn').copy()# and fit_to == "errors"')
    
    ute,fitto = bestCalcs.query('runname == @rn').iloc[0][['use_true_errors','fit_to']]
    
    row = desc_allsubj.loc[rn,cols]
    ss = '\n' + row2s(row)
    print(rn,ss)
    
    if len(df_) == 0:
        print('Emtpy for ',rn)
        continue
    with warnings.catch_warnings(record=True):
        warnings.filterwarnings("ignore",category=FutureWarning)
        warnings.filterwarnings("ignore",category=UserWarning)        
        fg,df__ = relplot_multi(sep_ys_by = 'hue', kind='line',data=df_,
                x='trial_index',
            ys=ys, 
            ylabel = ['Error discrepancy [deg]','Error [deg]','Simulated error sensitivity'],
            ylim=[(-10,30), (-50,50), (-0.01,0.125)], aspect = 2, 
                                height = 3,
            facet_kws = {'sharex':True, 'sharey':False}, ci='sd')

        fg.refline(y=0)    
        fg.refline(x=192)
        fg.refline(x=192*2)
        fg.refline(x=192*3)
        fg.axes[2,0].axhline(y=row[('initial_sensitivity')]['mean'],
                            color='grey',ls='--')
        #if rn.find('u')
        ttl0 = f'Use true history = {bool(ute)}'
        #plt.suptitle(, fontsize=12)    
        plt.suptitle(ttl0 + '\n' + ss, fontsize=10)    

        for tr in trs:
            fg.refline(x=tr) 
                
                
        plt.savefig(pjoin(path_fig, '_Herz_fit_to_ES', rn + '.pdf'))
        print('   saved')
        #plt.close()
        
    del df__,df_
    
    print(rn)